# **Analyse en composantes principales sur les vidéos tendances de YouTube**

## Introduction
**YouTube** a su s'ancrer dans le paysage quotidien des jeunes générations ces dernières années. Ce média social, plateforme d'hébergement de vidéos, recense plusieurs centaines de millions de visiteurs chaque mois et a marqué un tournant majeur dans le domaine du divertissement.
Pour résumer brièvement le principe, les auteurs y publient des vidéos qui pourront être visionnées par tout le monde : chaque personne qui visionne l'oeuvre ainsi constituée apporte une **vue**. Il y a aussi possibilité de signaler son appréciation du contenu en laissant un **like**, ou à l'inverse, sa déception avec un **dislike**. Enfin, des messages peuvent être postés sous la vidéo, communément appelés **commentaires**, signe d'activité ou de débat au sein de la communauté des spectateurs.
YouTube possède aussi des catégories : nous nous intéresserons aussi aux **Tendances** à qui rassemblent les vidéos les plus regardées et mises en avant des derniers jours, afin d'avoir les éléments précedemment cités en nombre suffisant pour l'étude. Bien évidemment, cette catégorie fluctuant au fil des semaines, on se concentra essentiellement sur des oeuvres qui y sont autrefois passées dans la même période.

Nous allons donc ici observer et analyser ces données, afin de peut-être en tirer des corrélations et conclusions.

## La base de données
Source : [Trending YouTube Video Statistics](https://www.kaggle.com/datasnaek/youtube-new)

La dataset est formée des colonnes suivantes :
* *video_id* : identifiant YouTube de la vidéo
* *trending_date* : date du passage en tendances de la vidéo
* *title* : titre de la vidéo
* *channel_title* : titre de la chaîne YouTube qui a posté la vidéo
* *category_id* : catégorie de la vidéo
* *publish_time* : date de publication de la vidéo
* *tags* : étiquettes données à la vidéo
* *views* : nombre de vues de la vidéo
* *likes* : nombre de likes de la vidéo
* *dislikes* : nombre de dislikes de la vidéo
* *comment_count* : nombre de commentaires de la vidéo
* *thumbnail_link* : lien de l'image miniature de la vidéo
* *comments_disabled* : désactivation des commentaires de la vidéo
* *ratings_disabled* : désactivation des likes et dislikes de la vidéo
* *video_error_or_removed* : vidéo inaccessible ou retirée
* *description* : description de la vidéo

In [ ]:
# Librairies
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
# Récupération de la base de données
df = pd.read_csv("../input/FRvideos.csv")

# Élimination des lignes indésirables
df = df[df.comments_disabled == False] # On souhaite les commentaires activés
df = df[df.ratings_disabled == False] # On souhaite les likes et dislikes activés
df = df[df.video_error_or_removed == False] # On souhaite la vidéo active à l'époque du dataset
df = df.dropna() # On souhaite des données complètes

# Stockage du nombre d'individus
rows = df.shape[0]

# Premier affichage
df.head()

## Obtention de données centrées réduites
Avant d'entamer l'ACP, il nous faut sélectionner et préparer les données utilisées par la suite : on utilise pour cela une fonction prévue à cet effet.

In [ ]:
# On conserve les 4 colonnes qui nous intéressent
select = ['views', 'likes', 'dislikes', 'comment_count']

# On normalise la dataset ainsi conservée
dfnorm = df[select]
dfnorm = StandardScaler().fit_transform(dfnorm)

# On vérifie que la normalisation est un succès
print("Moyennes :\n", np.mean(dfnorm, axis = 0)) # Les moyennes devraient valoir 0 ou être infiniment petites avec l'arrondi
print("\nÉcarts-types :\n", np.std(dfnorm, axis = 0)) # Les écarts-types devraient valoir 1
print("\nMatrice de corrélation :\n", (1/rows) * np.matmul(np.transpose(dfnorm), dfnorm)) # La matrice doit avoir des 1 sur la diagonale

## L'Analyse en Composantes Principales
Nous entrons désormais dans le vif du sujet.
On utilisera par la suite des outils appropriés qui éviteront par exemple des manipulations fastidieuses de matrice.

In [ ]:
# ACP bénigne afin d'utiliser ses fonctions de calculus de valeurs propres et variances
acp = PCA()
acp.fit_transform(dfnorm)

# Éléments nécessaire au choix du nombre de composantes principales
print("Valeurs propres :\n", acp.explained_variance_)
print("\nQualité de représentation des axes en % :\n", acp.explained_variance_ratio_ * 100)
print("\nQualité cumulée en % :\n", np.cumsum(acp.explained_variance_ratio_) * 100)

On observera une contribution de **91.65%** lorsqu'on ne considère que les deux premiers axes : en plus d'être la méthode de représentation la plus facile, c'est une valeur de précision relativement correcte.
Ainsi nous réaliserons un **ACP à 2 composantes principales**.

In [ ]:
# On génère ici l'ACP sur laquelle nous allons travailler
n = 2 # Nombre de composantes principales
acp = PCA(n)
composantes = acp.fit_transform(dfnorm)

# On affiche les premiers résultats
print("Valeurs propres :\n", acp.explained_variance_) # On rappelle les valeurs propres conservées
print("\nMatrice de changement de base :\n", acp.components_) # La matrice de changement de base, obtenue normalement à partir des vecteurs propres
print("\nComposantes principales :\n", composantes) # Les composantes principales données par la matrice de changement de base
print("\nSomme des composantes :\n", composantes.sum(axis = 0)) # La somme par colonne des composantes principales doit valoir 0

## Qualité de la représentation de chaque individu


In [ ]:
# On observe les représentations des individus sur chaque axe
quality = composantes ** 2
for i in range(n):
   quality[:,i] = quality[:,i] / np.sum(dfnorm ** 2, axis=1)
print("Pourcentage de représentation des individus par axe :\n", quality * 100)

## Contribution des individus aux axes

In [ ]:
# On observe les contributions des individus aux axes
contrib = composantes ** 2
for i in range(n):
    contrib[:,i] = contrib[:,i] / (rows * acp.explained_variance_[i])

print("Contribution aux axes :\n", contrib)

## Représentation des individus
On regardera ici la proximité (ou l'éloignement) des 10 premiers individus du dataframe : on constate une certain distance entre les vidéos très visionnées et le reste.

In [ ]:
# Création des figures
fig, axes = plt.subplots(figsize=(12, 12))

# Sélectionner les 10 premiers éléments comme exemple
tmp = pd.DataFrame(columns=list(df))
for i in range(10):
    tmp.loc[i]=df.iloc[i]
    plt.annotate(tmp.loc[i].channel_title, (composantes[i, 0], composantes[i, 1]))
    
# Échelle des axes
plt.plot([-2,2], [0,0])
plt.plot([0,0], [-2,2])

# Affichage du graphique et du dataset
plt.show()
select2 = ['channel_title', 'views', 'likes', 'dislikes', 'comment_count']
tmp[select2]

## Représentation des variables
On trace maintenant le **cercle des corrélations** : nous conclurons ensuite sur les résultats observés.

In [ ]:
# Corrélations variables-facteurs
x = np.sqrt(acp.explained_variance_)
y = np.zeros((len(select), len(select)))
for i in range(n):
    y[:,i] = acp.components_[i,:] * x[i]

print(pd.DataFrame({'Variable':select, 'Composante 1':y[:,0], 'Composante 2':y[:,1]}))

In [ ]:
# Création des figures
fig, axes = plt.subplots(figsize=(12,12))

# Récupération des noms
for i in range(len(select)):
    plt.annotate(select[i],(y[i,0], y[i,1]))

# Échelle des axes
plt.plot([-1,1],[0,0])
plt.plot([0,0],[-1,1])

# Ajouter un cercle
cercle = plt.Circle((0,0), 1, fill=False)
axes.add_artist(cercle)

# Affichage du cercle
plt.show()

## Conclusion
Sans grande surprise, les **vues** et le **nombre de commentaires** semblent assez **corrélés** : c'est assez logique, puisque les personnes réagiront à une vidéo et y répondront uniquement s'ils la voient.
Les **likes** ne sont **guère loin** : si une vidéo est très regardée, il risque d'y avoir des raisons derrière qui expliquent son succès et donc la potentielle quantité de likes qui en découle.
Les **dislikes** par contre sont plus **éloignés** : on conviendra du fait qu'une vidéo, populaire ou non, peut décevoir ou être détestée simplement de par son contenu, relativement indépendemment du reste.